In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import uniform_filter1d

from pyplume import plotting, utils
from pyplume.dataloaders import SurfaceGrid, DataLoader

In [ ]:
targets = {
    # "Tijuana OI": "data/field_netcdfs/oi_fields/Tot_SDLJ_202209.nc",
    # "Tijuana UWLS": "data/field_netcdfs/tj_plume_1km_2022-09_interped.nc",
    "New York/Jersey HFRNet": "data/field_netcdfs/hurrhenri_hfrnet.nc",
    "New York/Jersey HYCOM": "data/field_netcdfs/hurrhenri_hycom.nc",
}

In [ ]:
grids = {}
for t in targets:
    ds = DataLoader(targets[t]).dataset
    grids[t] = SurfaceGrid(ds)

In [ ]:
domain = {
    "S": 40,
    "N": 41,
    "W": -74.5,
    "E": -73.2,
}
for k, g in grids.items():
    plotting.plot_vectorfield(g.dataset, domain=domain)

In [ ]:
# pos = [32.6, -117.17]
pos = [40.5, -73.85]
# timerng = ["2022-09-08T06:00", "2022-09-12T00:00"]
timerng = ["2021-08-21T12:00", "2021-08-23T18:00"]

timerng = [np.datetime64(t) for t in timerng]

notes:

tijuana coord: 32.59638977050781, -117.00281524658203

new york bay coord: 40.5, -73.85

In [ ]:
fig, axs = plt.subplots(2, 1)
fig.set_size_inches(7.5, 7)
# hack to cycle the default line colors forward
num_col_skip = 2
for _ in range(num_col_skip):
    axs[0].plot([], [])
    axs[1].plot([], [])
for k, g in grids.items():
    idx = g.get_closest_index(t=None, lat=pos[0], lon=pos[1])
    print(
        f"found closest coordinate for {k} at {(g.dataset['lat'][idx[1]].values.item(), g.dataset['lon'][idx[2]].values.item())}"
    )
    sliced = g.dataset.isel(lat=idx[1], lon=idx[2]).sel(
        time=slice(timerng[0], timerng[1])
    )
    axs[0].plot(sliced["time"], sliced["U"], label=k)
    axs[1].plot(sliced["time"], sliced["V"], label=k)
    # 36 hour running average specifically for NY HFRNet readings
    if k == "New York/Jersey HFRNet":
        axs[0].plot(sliced["time"], uniform_filter1d(sliced["U"], size=12, mode="nearest", origin=5), "--", label=f"{k} 12 hr avg")
        axs[1].plot(sliced["time"], uniform_filter1d(sliced["V"], size=12, mode="nearest", origin=5), "--", label=f"{k} 12 hr avg")
for ax in axs:
    ax.grid()
    ax.legend()
    for tick in ax.get_xticklabels():
        tick.set_rotation(20)
axs[0].set_title("Vector zonal component")
axs[0].set_ylabel("m/s")
axs[1].set_title("Vector meridional component")
axs[1].set_ylabel("m/s")
# fig.suptitle("Tijuana vector timeseries", fontsize=16)
fig.suptitle("New York Bay vector timeseries", fontsize=16)
fig.tight_layout()
fig.patch.set_facecolor("w")